In [32]:
import pandas as pd
import numpy as np
import re
import docx

In [39]:
df = pd.read_excel('authors3.xlsx', dtype=str)
df = df.fillna("nan")
df.head(1)

,First Name,Last Name,Email,Affiliation,Department,City,Postcode,Country of Affiliation,OrcidID,"Contribution type (1=data collection, 2=intervention designer, 3=funding)",Funding (source of findings used for this project),Grant Number (if applicable),Conflict of Interest,@dropdown,Comments,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,Madalina,Vlasceanu,mov209@nyu.edu,New York University,Department of Psychology,New York,10003,USA,https://orcid.org/0000-0003-2138-1968,nan,Google Jigsaw,nan,No,nan,nan,nan,nan,nan,nan,nan


In [40]:
df["Full Name"] = df["First Name"] + " " + df["Last Name"]


In [41]:
df["Full Affiliation"] = df["Department"]  + ", " + df["Affiliation"] + "; " + df["City"] + ", "  + df["Postcode"] + ", " + df["Country of Affiliation"] + "."


In [42]:
authors = {}
affiliations = np.array([])
for name, author in df.groupby("Full Name", sort=False):
    nums = []
    for a in author["Full Affiliation"]:
        if a not in affiliations:
            affiliations = np.append(affiliations, a)
        nums.append(np.where(affiliations == a)[0][0] + 1)
    authors[name] = ",".join(map(str, sorted(nums)))
print(authors)
print(affiliations)

{'Madalina Vlasceanu': '1', 'Kimberly C. Doell': '1,2', 'Joseph B. Bak Coleman': '3,4,5', 'Boryana Todorova': '2', 'Michael M. Berkebile-Weinberg': '1', 'Samantha J. Grayson': '6', 'Yash Patel': '1', 'Danielle Goldwert': '1', 'Yifei Pei': '1', 'Alek Chakroff': '7', 'Ekaterina Pronizius': '2', 'Karlijn L. van den Broek': '8', 'Denisa Vlasceanu': '9', 'Sara Constantino': '10,11', 'Philipp Schumann': '12', 'Steve Rathje': '1', 'Salvatore Maria Aglioti': '13,14', 'Mark Alfano': '15', 'Andy J. Alvarado-Yepez': '16', 'Angélica Andersen': '17', 'Frederik Anseel': '18', 'Matthew A. J. Apps': '19', 'Chillar Asadli': '20', 'Fonda Jane Awuor': '21', 'Flavio Azevedo': '22', 'Piero Basaglia': '23', 'Jocelyn J. Bélanger': '24', 'Sebastian Berger': '25', 'Paul Bertin': '26,27', 'Michał Białek': '28', 'Olga Bialobrzeska': '29', 'Michelle Blaya-Burgo': '30', 'Daniëlle N. M. Bleize': '31', 'Simen Bø': '32', 'Lea Boecker': '33', 'Paulo S. Boggio': '34', 'Sylvie Borau': '35', 'Björn Bos': '36', 'Ayoub Bou

/tmp/ipykernel_63357/2829855779.py:6: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if a not in affiliations:


In [43]:
from docx import Document
document = Document()

p = document.add_paragraph()
for name, affilnums in authors.items():    
    _ = p.add_run(name)
    sup = p.add_run(affilnums)
    sup.font.superscript = True
    _ = p.add_run(f", ")    

for i, affil in enumerate(affiliations):
    p = document.add_paragraph()
    sup = p.add_run(f"{i+1}")
    sup.font.superscript = True
    _ = p.add_run(f"{affil}")
    

# for author in df.rows():
#     super_text = p.add_run('superscript text')
#     super_text.font.superscript = True

#     p.add_run(' and ')

#     sub_text = p.add_run('subscript text')
#     sub_text.font.subscript = True

document.save('authors.docx')

---

In [38]:
# import itertools
# unique_affiliations = set(itertools.chain.from_iterable(df.SimpleAffiliation))

lookup = np.array([])
for i, author in df.iterrows():
    nums = []
    for affil in author.SimpleAffiliation: # If none, skips
        if affil not in lookup:
            lookup = np.append(lookup, affil)
        nums.append(np.where(lookup == affil)[0][0]+1)
    if len(nums) > 0:
        df.iloc[i]["NumAffiliation"] = ",".join(map(str, sorted(nums)))


AttributeError: 'Series' object has no attribute 'SimpleAffiliation'

In [4]:
from docx import Document
document = Document()

p = document.add_paragraph()
for i, author in df.iterrows():    
    _ = p.add_run(f"{author.First} {author.Last}")
    sup = p.add_run(f"{author.NumAffiliation}")
    sup.font.superscript = True
    _ = p.add_run(f", ")    

for i, affil in enumerate(lookup):
    p = document.add_paragraph()
    sup = p.add_run(f"{i+1}")
    sup.font.superscript = True
    _ = p.add_run(f"{affil}")

p = document.add_paragraph()    
sup = p.add_run("-1")
sup.font.superscript = True
_ = p.add_run(f"TKTK")
    

# for author in df.rows():
#     super_text = p.add_run('superscript text')
#     super_text.font.superscript = True

#     p.add_run(' and ')

#     sub_text = p.add_run('subscript text')
#     sub_text.font.subscript = True

document.save('authors.docx')